In [4]:
!pip install -q drawdata

In [5]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

import ipywidgets
from sklearn.inspection import DecisionBoundaryDisplay
from IPython.display import HTML
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBClassifier

from drawdata import ScatterWidget

In [8]:
output = ipywidgets.Output()
widget = ScatterWidget()

classifier_selector = ipywidgets.RadioButtons(
    options=[
        'Logistic Regression', 'Random Forest', 'SVC Poly', 'SVC RBF',
        'KNN', 'Decision Tree', 'Naive Bayes', 'XGBoost', 'AdaBoost'
    ],
    description='Classifier:',
)

@output.capture(clear_output=True)
def on_change(change):
    df = widget.data_as_pandas
    if len(df) and (df['color'].nunique() > 1):
        X = df[['x', 'y']].values
        y = df['color']

        # تبدیل برچسب‌های رشته‌ای به اعداد
        le = LabelEncoder()
        y_encoded = le.fit_transform(y)

        display(HTML("<br><br><br>"))
        fig = plt.figure(figsize=(12, 12))

        if classifier_selector.value == 'Logistic Regression':
            classifier = LogisticRegression().fit(X, y_encoded)
        elif classifier_selector.value == 'SVC Poly':
            classifier = SVC(kernel='poly').fit(X, y_encoded)
        elif classifier_selector.value == 'SVC RBF':
            classifier = SVC(kernel='rbf').fit(X, y_encoded)
        elif classifier_selector.value == 'Random Forest':
            classifier = RandomForestClassifier().fit(X, y_encoded)
        elif classifier_selector.value == 'KNN':
            classifier = KNeighborsClassifier().fit(X, y_encoded)
        elif classifier_selector.value == 'Decision Tree':
            classifier = DecisionTreeClassifier().fit(X, y_encoded)
        elif classifier_selector.value == 'Naive Bayes':
            classifier = GaussianNB().fit(X, y_encoded)
        elif classifier_selector.value == 'XGBoost':
            classifier = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss').fit(X, y_encoded)
        elif classifier_selector.value == 'AdaBoost':
            classifier = AdaBoostClassifier().fit(X, y_encoded)

        disp = DecisionBoundaryDisplay.from_estimator(
            classifier,
            X,
            response_method="predict",
            xlabel="x", ylabel="y",
            alpha=0.5,
        )
        disp.ax_.scatter(X[:, 0], X[:, 1], c=y_encoded, edgecolor="k", cmap='coolwarm')
        plt.title(f"{classifier.__class__.__name__}")
        plt.show()

widget.observe(on_change, names=["data"])
classifier_selector.observe(on_change, names="value")
on_change(None)

ipywidgets.HBox([ipywidgets.VBox([widget, classifier_selector]), output])
